In [36]:
file=open("blogdata.txt","r")
lines=[line for line in file]

#取列标题
colnames=lines[0].strip().split('\t')[1:]
# 每行的index
rownames=[]
# 每行的数据
data=[]
for line in lines[1:]:
    p=line.strip().split('\t')
    rownames.append(p[0])
    data.append([float(x) for x in p[1:]])
print(rownames)
print(colnames)

print(data)

["The Superficial - Because You're Ugly", 'Wonkette', 'Publishing 2.0', 'Eschaton', 'Blog Maverick', 'Mashable!', 'we make money not art', 'GigaOM', 'Joho the Blog', "Neil Gaiman's Journal", 'Signal vs. Noise', 'lifehack.org', 'Hot Air', 'Online Marketing Report', 'Kotaku', 'Talking Points Memo: by Joshua Micah Marshall', "John Battelle's Searchblog", '43 Folders', 'Daily Kos', 'Deadspin', 'Go Fug Yourself', "O'Reilly Radar", 'Andrew Sullivan | The Daily Dish', 'Lifehacker', 'Google Operating System', 'Valleywag', 'Gizmodo', 'ScienceBlogs : Combined Feed', 'Michelle Malkin', 'SimpleBits', 'Slashdot', 'Gothamist', 'BuzzMachine', "Sifry's Alerts", 'Topix.net Weblog', 'The Viral Garden', 'Micro Persuasion', 'Cool Hunting', 'SpikedHumor', 'flagrantdisregard', 'Search Engine Watch Blog', 'Joystiq', 'Download Squad', "Captain's Quarters", 'MAKE Magazine', 'Engadget', 'CoolerHeads Prevail', 'Power Line', 'Techdirt', 'Official Google Blog', 'The Blotter', 'Treehugger', 'Crooks and Liars', 'TMZ

In [2]:
# 计算两个向量的相关度分值
from math import sqrt
def pearson(v1,v2):
    sum1=sum(v1)
    sum2=sum(v2)
    
    sum1Sq=sum([pow(v,2) for v in v1])
    sum2Sq=sum([pow(v,2) for v in v2])
    
    pSum=sum([v1[i] * v2[i] for i in range(len(v1))])
    
    num=pSum-(sum1*sum2/len(v1))
    den=sqrt((sum1Sq-pow(sum1,2)/len(v1))*(sum2Sq-pow(sum2,2)/len(v1)))
    
    if den==0:return 0
    return 1.0-num/den
pearson([1,2,3],[1,2,3])

0.0

In [3]:
# 树的结点
class bicluster:
    def __init__(self,vec,left=None,right=None,distance=0.0,id=None):
        self.left=left
        self.right=right
        self.vec=vec
        self.id=id
        self.distance=distance

In [25]:
def hcluster(rows,distance=pearson):
    distances={}
    currentclusterid=-1
#     对每一行都生成一个对象
    clust=[bicluster(rows[i],id=i) for i in range(len(rows))]
    
    while len(clust)>1:
        lowestpair=(0,1)
        closest=distance(clust[0].vec,clust[1].vec)
        
#         遍历每一个配对
        for i in range(len(clust)):
            for j in range(i+1,len(clust)):
                if(clust[i].id,clust[j].id) not in distances:
#                     distance来存距离
                    distances[(clust[i].id,clust[j].id)] =distance(clust[i].vec,clust[j].vec)
                d=distances[(clust[i].id,clust[j].id)]
        
                if d<closest:
                    closest=d
                    lowestpair=(i,j)
                    
#         计算两个聚类的平均值
        mergevec=[(clust[lowestpair[0]].vec[i]+clust[lowestpair[1]].vec[i])/2.0 for i in range(len(clust[0].vec))]
    
#      建立新的聚类
        newcluster=bicluster(mergevec,left=clust[lowestpair[0]],right=clust[lowestpair[1]],distance=closest,id=currentclusterid)
    
        currentclusterid=-1
        del clust[lowestpair[1]]
        del clust[lowestpair[0]]
        clust.append(newcluster)
    return clust[0]        


In [26]:
clust=hcluster(data)

In [30]:
print(clust.vec)

[0.48046875, 0.62109375, 1.55078125, 1.1650390625, 2.9951171875, 0.724609375, 1.1689453125, 0.54296875, 0.708984375, 0.701171875, 4.056640625, 0.384765625, 1.296875, 0.4873046875, 1.8017578125, 0.982421875, 1.181640625, 0.5537109375, 0.865234375, 0.6171875, 1.017578125, 1.9296875, 0.78515625, 0.9765625, 0.7412109375, 1.9169921875, 0.380859375, 0.9306640625, 0.40234375, 1.9609375, 0.494140625, 0.4755859375, 0.6220703125, 0.572265625, 0.3203125, 0.4765625, 0.7734375, 3.7431640625, 1.0654296875, 2.2216796875, 0.828125, 0.642578125, 3.0068359375, 0.814453125, 0.5888671875, 2.9931640625, 0.6904296875, 0.88671875, 0.55078125, 0.677734375, 0.4560546875, 0.67578125, 1.875, 0.87890625, 0.9375, 0.8017578125, 1.048828125, 0.876953125, 0.3984375, 0.9716796875, 1.04296875, 0.5732421875, 1.0966796875, 2.1923828125, 0.40234375, 0.4013671875, 2.41796875, 0.3212890625, 0.486328125, 0.412109375, 1.009765625, 0.65625, 0.7646484375, 0.8134765625, 0.662109375, 0.798828125, 0.544921875, 0.931640625, 0.41601

In [39]:
# 打印聚类结果
def printclust(clust,labels=None,n=0):
    for i in range(n):
        print(' ',)
    if clust.id<0:
        print('-')
    else:
        if labels==None:
            print(clust.id)
        else:
            print(labels[clust.id])
    if clust.left != None:
        printclust(clust.left,labels=labels,n=n+1)
    if clust.right !=None:
        printclust(clust.right,labels=labels,n=n+1)

In [40]:
printclust(clust,rownames)

-
 
-
 
 
-
 
 
 
-
 
 
 
 
-
 
 
 
 
 
-
 
 
 
 
 
 
Techdirt
 
 
 
 
 
 
Creating Passionate Users
 
 
 
 
 
-
 
 
 
 
 
 
Blog Maverick
 
 
 
 
 
 
Joel on Software
 
 
 
 
-
 
 
 
 
 
-
 
 
 
 
 
 
BuzzMachine
 
 
 
 
 
 
Bloggers Blog: Blogging the Blogsphere
 
 
 
 
 
-
 
 
 
 
 
 
Topix.net Weblog
 
 
 
 
 
 
Steve Pavlina's Personal Development Blog
 
 
 
-
 
 
 
 
-
 
 
 
 
 
-
 
 
 
 
 
 
PaulStamatiou.com
 
 
 
 
 
 
-
 
 
 
 
 
 
 
A Consuming Experience (full feed)
 
 
 
 
 
 
 
-
 
 
 
 
 
 
 
 
Official Google Blog
 
 
 
 
 
 
 
 
-
 
 
 
 
 
 
 
 
 
Joystiq
 
 
 
 
 
 
 
 
 
The Unofficial Apple Weblog (TUAW)
 
 
 
 
 
-
 
 
 
 
 
 
Mashable!
 
 
 
 
 
 
Slashdot
 
 
 
 
-
 
 
 
 
 
-
 
 
 
 
 
 
we make money not art
 
 
 
 
 
 
Joi Ito's Web
 
 
 
 
 
-
 
 
 
 
 
 
Copyblogger
 
 
 
 
 
 
Oilman
 
 
-
 
 
 
-
 
 
 
 
-
 
 
 
 
 
-
 
 
 
 
 
 
lifehack.org
 
 
 
 
 
 
Dave Shea's mezzoblue
 
 
 
 
 
-
 
 
 
 
 
 
SpikedHumor
 
 
 
 
 
 
MetaFilter
 
 
 
 
-
 
 
 
 
 
-